In [146]:
import sumolib 
import traci
from sumolib import checkBinary

In [147]:
junction = {'KlongTei': 'cluster_1892287670_272491964_272492178',
      'Rama4': 'cluster_272488163_282390730_66263210_66263222',
      'NaRanong': 'cluster_272488164_272492179_3457051443_61907354',
      'Sunthon': 'gneJ83',
      'KasemRat': 'cluster_272448137_272555800_272555808_7660045934_7710268409',
      'Virtual' : '270329335'}
direction = {'KlongTei' : ['NB','WB','EB'], 
             'Rama4': ['WB','SB','EB','NB'],
             'NaRanong': ['NB','WB','SW','SB','EB'],
             'Sunthon': ['WB','SB','EB','NB'],
             'KasemRat': ['WB','EB','NB'],
             'Virtual' : ['SB','EB']}

In [148]:
# KlongTei = 'cluster_1892287670_272491964_272492178'
# Rama4 = 'cluster_272488163_282390730_66263210_66263222'
# NaRanong = 'cluster_272488164_272492179_3457051443_61907354'
# Sunthon = 'gneJ83'
# KasemRat = 'cluster_272448137_272555800_272555808_7660045934_7710268409'
# Virtual = '270329335'
# junction = [KlongTei,Rama4,NaRanong,Sunthon,KasemRat,Virtual]

In [149]:
def reset():
    sumoBinary = checkBinary('sumo-gui')
    traci.start([sumoBinary, "-c", "Kasemrat-SUMO-UsingBookNetFile/osm.sumocfg",
                             "--tripinfo-output", "tripinfo.xml", '--start','true','--quit-on-end','true','--time-to-teleport','-1'])

In [150]:
net1 = sumolib.net.readNet('Kasemrat-SUMO-UsingBookNetFile/AddLane.net.xml')

In [195]:
def get_connect(ne):
    dir_con = set()
    out = ne.getToNode().getOutgoing()
    for o in out:
#                                 print(o)
        con = sumolib.net.edge.Edge.getConnections(ne,o)
#                                 print(con)
        for c in con:
            dir_con.add(sumolib.net.connection.Connection.getDirection(c).upper())
    connect = ''
    if 'L' in dir_con : connect += 'L'
    else: connect += 'X'
    if 'S' in dir_con : connect += 'S'
    else: connect += 'X'
    if 'R' in dir_con : connect += 'R'
    else: connect += 'X'
    if 'U' in dir_con : connect += 'U'
    else: connect += 'X'
    return connect

In [153]:
reset()

In [152]:
traci.close()

In [225]:
lane_name = {}
last_edge = None
NoNeed = ['27702347#6', '300813736#0', '825786399','459492913#3', '459492913#2','459492913#0','618739073#1',
          '825786398', '482384842#2', '751759907']
for j in junction.keys(): 
    if j == 'Rama4':
        dir_index = -1
        EdgeIndex = 1
        lane_1 = list(traci.trafficlight.getControlledLanes(junction[j]))
        for l_1 in lane_1:
            if l_1[-1] == '0' and l_1[:-2] != last_edge:
                EdgeIndex = 1
                dir_index += 1
                edge = l_1[:-2]
                for i in range(10):
                    nextEdges = net1.getEdge(edge).getFromNode().getIncoming()
                    for ne in nextEdges:
                        if ne.getLaneNumber() > 1 and ne.getID() not in NoNeed:
                            connect = get_connect(ne)
    #                         print(connect)
                            EdgeIndex += 1
                            edgeID = ne
                            numOfLane = edgeID.getLaneNumber()
                            for LaneIndex in range(numOfLane):
                                nextLanes = edgeID.getID()+'_'+str(LaneIndex)
                                if 'R' in connect and LaneIndex == numOfLane-1:
                                    lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(EdgeIndex)+'_'+str(LaneIndex)+'_'+connect] = nextLanes
                                elif 'L' in connect and LaneIndex == 0:
                                    lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(EdgeIndex)+'_'+str(LaneIndex)+'_'+connect] = nextLanes

                                else:
                                    lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(EdgeIndex)+'_'+str(LaneIndex)+'_'+'XSXX'] = nextLanes
                            edge = edgeID.getID()
            LaneIndex = int(l_1[-1])
            connect = get_connect(net1.getEdge(l_1[:-2]))
#             print(connect)
            numOfLane = net1.getEdge(l_1[:-2]).getLaneNumber()
            if 'R' in connect and LaneIndex == numOfLane-1 and 'U' not in connect:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'XSRX'] = l_1
            elif 'R' in connect and LaneIndex == numOfLane-1 and 'U' in connect:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'XSRU'] = l_1
            elif 'L' in connect and LaneIndex == 0:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'LSXX'] = l_1
            else:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'XSXX'] = l_1
            last_edge = l_1[:-2]
for l in sorted(lane_name):
# for l in lane_name:
    print(l, '=', "'"+lane_name[l]+"'")

RAMA4_EB_10_0_XSXX = '825786383#0_0'
RAMA4_EB_10_1_XSXX = '825786383#0_1'
RAMA4_EB_10_2_XSXX = '825786383#0_2'
RAMA4_EB_10_3_XSXX = '825786383#0_3'
RAMA4_EB_11_0_LSXX = '480596367#3_0'
RAMA4_EB_11_1_XSXX = '480596367#3_1'
RAMA4_EB_12_0_XSXX = '480596358#1_0'
RAMA4_EB_12_1_XSXX = '480596358#1_1'
RAMA4_EB_13_0_XSXX = '480596367#1_0'
RAMA4_EB_13_1_XSXX = '480596367#1_1'
RAMA4_EB_14_0_XSXX = '480596367#0_0'
RAMA4_EB_14_1_XSXX = '480596367#0_1'
RAMA4_EB_1_0_XSXX = '825786410_0'
RAMA4_EB_1_1_XSXX = '825786410_1'
RAMA4_EB_1_2_XSXX = '825786410_2'
RAMA4_EB_1_3_XSRX = '825786410_3'
RAMA4_EB_2_0_LSXX = '459492917#0_0'
RAMA4_EB_2_1_XSXX = '459492917#0_1'
RAMA4_EB_2_2_XSXX = '459492917#0_2'
RAMA4_EB_2_3_XSXX = '459492917#0_3'
RAMA4_EB_2_4_XSXX = '459492917#0_4'
RAMA4_EB_3_0_LXRX = '25047974#2_0'
RAMA4_EB_3_1_XSXX = '25047974#2_1'
RAMA4_EB_3_2_XSXX = '25047974#2_2'
RAMA4_EB_3_3_LXRX = '25047974#2_3'
RAMA4_EB_4_0_LSXX = '481971011#1_0'
RAMA4_EB_4_1_XSXX = '481971011#1_1'
RAMA4_EB_4_2_XSXX = '4819710